# Tópicos avanzados de programación con Julia

## _Introducción a Shell Scripting_

__Recomendaciones generales y buenas prácticas al crear el entorno de desarrollo.__
- Introducción a Shell Scripting.
  - Crear y ejecutar cualquier script desde Jupyter.
  - Interacción con el usuario.
    - Entrada de datos.
    - Salida de datos.
  - Comandos externos, internos y palabras claves.
  - Arreglos
  - Comprobaciones en Bash.
    - Comandos `test`,  `[` y `[[`.
    - Cadenas.
    - Enteros.
    - Archivos.
    - Concatenando pruebas.
  - Estructuras de control de flujo.
    - `if`.
    - `case`.
    - `for`.
    - `while`.
    - `until`.
    - `select` estructura de control y de entrada de datos.
  - Diálogos interactivos desde la linea de comando.
- Miniproyecto (Instalador para el taller).

### Crear y ejecutar cualquier script desde Jupyter

Para agilizar el proceso y no estar cambiando de programa. Voy a mostrar la forma en la que se crearán los script desde aquí:
Supongamos que queremos crear un script en el directorio `./script` con el siguiente contenido:

```bash
#!/usr/bin/env bash
echo -e "El directorio actual es:\n\n $(pwd)"
```
Y además queremos darle permiso de ejecución, para no estar invocando constantemente al intérprete `bash`.

Un forma bastante artesanal sería:

In [ ]:
cd ./script #Nos movemos al subdirectorio «script», para que la ruta sea más corta.

In [ ]:
cat << "fin" > "./scp1.sh" #Creamos el script.
#!/usr/bin/env bash
echo -e "El directorio actual es:\n\n $(pwd)"
fin

chmod -x ./scp1.sh

cat ./scp1.sh #Revisamos, el script creado.

Podemos usar variables y funciones para mejorar la creación de scrips.

In [ ]:
#La función «scp» crea scripts ejecutables para bash en el directorio actual.
#El parámetro $1, contiene el nombre del archivo.
#Por defecto se añade la extensión «sh» y se le coloca el SheBang.
#Finalmente se presenta un informe de las tareas realizadas.
scp(){
    echo '#!/usr/bin/env bash' > "./$1.sh" #Crea el archivo con SheBang.
    #cat espera que se añadan las lineas manualmente.
    #Obviamente, podemos redirigir la entrada de «scp» que es lo mismo
    #que haberlo hecho directamente con cat.
    cat >>"./$1.sh" #Todas las lineas se agregan después del SheBang.
    chmod +x "./$1.sh" #Le damos permiso de ejecución.
    
    #Se crea el informe:   
    echo "Se creó el archivo ejecutable «$1.sh»."
    echo -e "\nEl archivo «$1.sh» está contenido en:\n\n$(pwd)\n"
    echo "El contenido de «$1.sh» es:"
    echo -e "---------------------------\n"
    cat "./$1.sh"
    echo -e "\n---------------------------"
    echo "Fin del contenido de «$1.sh»"
}

Ahora todo es mucho más fácil.

In [ ]:
scp scp2 << "fin"
echo -e "El directorio actual es:\n\n $(pwd)"
fin

In [ ]:
#La función «scp» crea scripts ejecutables para bash en el directorio actual.
#El parámetro $1, contiene el nombre del archivo.
#Por defecto se añade la extensión «sh» y se le coloca el SheBang.
#Con la opción «-i» o «--info» se presenta un informe de las tareas realizadas.
scp(){
    local informe=false #Para saber si se muestra el informe.
    
    #Si está la opción de imprimir informe
    if [[ $1 = "-i" ||  $1 = "--info" ]]; then
        informe=true #Para indicar que se va a imprimir.
        shift #Se elimina el parámetro de tipo opción.
    elif [[ $1 =~ ^- ]]; then #Cualquier otra opción será incorrecta.
        echo "Opción «$1» incorrecta."
        return 1
    fi
    
    #En este punto el nombre del archivo está en $1.
    #Si $1 está vacío, se notifica y retornamos un error 1.
    if [[ -z "$1" ]]; then
        echo "Nombre de archivo vacío."
        return 1
    fi
    
    echo '#!/usr/bin/env bash' > "./$1.sh" #Crea el archivo con SheBang.
    #cat espera que se añadan las lineas manualmente.
    #Obviamente, podemos redirigir la entrada de «scp» que es lo mismo
    #que haberlo hecho directamente con cat.
    cat >>"./$1.sh" #Todas las lineas se agregan después del SheBang.
    chmod +x "./$1.sh" #Le damos permiso de ejecución.
    
    #Si «informe» es verdadero, este se crea:
    if $informe; then
        echo "Se creó el archivo ejecutable «$1.sh»."
        echo -e "\nEl archivo «$1.sh» está contenido en:\n\n$(pwd)\n"
        echo "El contenido de «$1.sh» es:"
        echo -e "---------------------------\n"
        cat "./$1.sh"
        echo -e "\n---------------------------"
        echo "Fin del contenido de «$1.sh»"
    fi
}

In [ ]:
scp -i prueba << "fin"
echo -e "El directorio actual es:\n\n $(pwd)"
fin

In [ ]:
scp --info prueba2 << "fin"
echo -e "El directorio actual es:\n\n $(pwd)"
fin

In [ ]:
scp --ifo prueba3 << "fin"
echo -e "El directorio actual es:\n\n $(pwd)"
fin

In [ ]:
scp -i << "fin"
echo -e "El directorio actual es:\n\n $(pwd)"
fin

In [ ]:
scp << "fin"
echo -e "El directorio actual es:\n\n $(pwd)"
fin

Podemos ejecutar el script creado en otra terminal, sin salir de Jupyter.

In [ ]:
x-terminal-emulator -e './scp2.sh' #Abre una nueva terminal, ejecuta el script y shttp://localhost:8889/notebooks/Pl%C3%A1ticas/Pl%C3%A1tica-9-Introducci%C3%B3n-a-Shell-Scripting-part-3/Pl%C3%A1tica-9-Introducci%C3%B3n-a-Shell-Scripting-part-3.ipynb#e cierra.

In [ ]:
#Abre una nueva terminal, ejecuta el script. Espera a que se cierre manualmente.
x-terminal-emulator --noclose -e './scp2.sh'

In [ ]:
#Podemos crear un alias para no tener que escribir tanto.
alias exc="x-terminal-emulator --noclose -e"

In [ ]:
exc ./scp2.sh #Mucho más cómodo.

In [ ]:
unalias exc #Eliminamos el alias, para crear un función más cómoda aún.

In [ ]:
#Hace lo mismo, pero evita tener que colocar la ruta relativa y la extensión.
exc(){
    x-terminal-emulator --noclose -e "./$1.sh"
}

In [ ]:
exc scp2 #Aún más cómodo.

In [ ]:
#Podemos usar una variable, para evitar escribir dos veces el nombre del archivo:
n_scp="scp3" #Ahora sólo tenemos que escribir el nombré aquí.

scp $n_scp << "fin"

echo -e "El directorio actual es:\n\n $(pwd)"

fin

exc $n_scp

### Interacción con el usuario

La interacción con el usuario en los scripts, se da mediante la entrada y la salida estándar de los procesos.

#### Entrada de datos

- Para obtener datos almacenados en algún archivo, podemos redirigir la entrada estándar del proceso que lo necesita al archivo en cuestión.
- Una manera alternativa, es usar comandos como `cat`, `head`, `tail`, etc.
- Los datos obtenidos de ese modo pueden pasarse a otros procesos o almacenarse en variables.
- Para obtener información directamente del usuario se usa el comando `read`.

In [ ]:
n_scp="scp4"

scp $n_scp << "fin"

#No da ningún aviso.
read nombre #Se lee desde el teclado y se almacena en «nombre»
echo "Su nombre es $nombre." #Muestra el contenido.

fin

exc $n_scp

In [ ]:
n_scp="scp5"

scp $n_scp << "fin"

echo "Escriba su nombre: "
read nombre #Se lee desde el teclado y se almacena en «nombre»
echo "Su nombre es $nombre." #Muestra el contenido.

fin

exc $n_scp

In [ ]:
n_scp="scp6"

scp $n_scp << "fin"

echo -n "Escriba su nombre: "
read nombre #Se lee desde el teclado y se almacena en «nombre»
echo "Su nombre es $nombre." #Muestra el contenido.

fin

exc $n_scp

In [ ]:
n_scp="scp7"

scp $n_scp << "fin"

read -p "Escriba su nombre: " nombre #Se lee desde el teclado y se almacena en «nombre»
echo "Su nombre es $nombre." #Muestra el contenido.

fin

exc $n_scp

#### Salida de datos

El comando `echo` es bastante práctico y difundido, pero no es recomendado cuando se necesita dar un mejor formato al texto de salida. Para eso contamos con `printf` (usa una sintaxis similar a la función `printf` del lenguaje `C`). 

[Aquí](https://ugeek.github.io/blog/post/2019-02-08-no-todo-es-echo-tambien-existe-printf.html) se muestran algunos ejemplos.

### Comandos externos, internos y palabras claves.

In [ ]:
type echo #Comando definido dentro de bash.

In [ ]:
type julia #Comando externo a bash.

Una palabra clave es una definición interna de __bash__. Son componentes básicos de su sintaxis, pero no son programas ( _comandos_ ). Necesitan de comandos para que tengan sentido. En realidad se comportan como operadores.

In [ ]:
type { #Palabra clave.

En [este](https://ubuntu.dokry.com/5924/cual-es-la-diferencia-entre-shell-builtin-y-shell-keyword.html) artículo, se explica bastante bien la diferencia entre los tres tipos de definiciones que usa __bash__.

### Arreglos

In [ ]:
# Los elementos de una arreglo se separan con espacios.
arreglo=(a b c 1 hola "nada de nada")

In [ ]:
echo $arreglo #Solo nos da el primer elemento ¿?

In [ ]:
var=12 #También es un arreglo, pero de un solo elemento.

In [ ]:
# Para ver los elemento del arreglo:
echo ${arreglo[@]} #Muestra todos los elementos separados por espacios.
echo ${arreglo[*]} #Muestra lo mismo, pero se trata al conjunto como un solo elemento.
#La variable también es un arreglo
echo ${var[@]}
echo ${var[*]}

In [ ]:
#Podemos seleccionar cada elemento de un arreglo.
echo ${arreglo[0]}
echo ${arreglo[1]}
echo ${arreglo[2]}
echo ${arreglo[3]}
echo ${arreglo[4]}
echo ${arreglo[5]}
#Como no existe, está vacío. Igual que todas las variables no declaradas.
echo ${arreglo[6]}
echo ${var[1]}

In [ ]:
#Se puede cambiar el valor de cualquier elemento o añadir nuevos:
arreglo[6]=NUEVO
var[1]=NUEVO
arreglo[2]=CAMBIO
#También podemos añadir elementos por delante:
var=(Anterior ${var[*]}) #Lo redefinimos y listo.
#Al redefinirlo de esta manera, puede haber problemas con los espacios.
echo ${arreglo[*]}
echo ${var[*]}

In [ ]:
a=23; b=44
#Con el signo de «!», miramos dentro de los elementos del array.
#Ahora si que tiene sentido lo de almacenar variables en variables.
echo ${!arreglo[0]}
echo ${!arreglo[1]}

In [ ]:
#Para saber cuantos elementos tiene un array:
echo ${#arreglo[@]}
echo ${#arreglo[*]}
echo ${#arreglo[2]} #Esto cuenta los elemento de los elementos (número de caracteres).

In [ ]:
#Para copiar un arreglo en otro:
arreglo2=(${arreglo[@]})
arreglo3=(${arreglo[*]})

#Se separó la cadena.
echo ${#arreglo2[@]}
echo ${#arreglo2[*]}
echo ${#arreglo3[@]}
echo ${#arreglo3[*]}
echo ${arreglo3[*]}

arreglo2=("${arreglo[@]}")
arreglo3=("${arreglo[*]}")

In [ ]:
#De esta manera se evita el problema.
declare -n arreglo4=arreglo
echo ${#arreglo4[@]}
echo ${#arreglo4[*]}
echo ${arreglo4[*]}

In [ ]:
arreglo5=(1 2 3 4 5 6 7 8 9) #Muestra todo.
#Se puede seleccionar sólo una parte.
echo ${arreglo5[@]:3:2} #Toma dos elementos a partir del cuarto.

echo ${arreglo5[@]:3} #Del cuarto en adelante.
echo ${arreglo5[@]::4} #Toma cuatro elementos a partir del primero.

In [ ]:
#Crea un rango de valores.
seq 1 5
echo "-------------"
seq 1 2 5
echo "-------------"
seq 5 -2 1

In [ ]:
#Se puede usar para crear arreglos
var2=$(seq 1 10) #Todavía no lo es.
echo ${var2[*]}
echo ${#var2[*]}

In [ ]:
#Se puede usar para crear arreglos
var2=($(seq 1 10)) #Ahora sí.
echo ${var2[*]}
echo ${#var2[*]}

Para más información ve la ayuda de `seq`.

También se pueden trabajar con otro tipo de rangos. Los veremos en la estructura `for`.

Este fue el ejemplo que mostraba el arreglo que almacena los estados de salida de los procesos unidos por tuberías.

In [ ]:
#El arreglo PIPESTATUS, almacena los estados de salida de cada comando.
cat /tmp/arch1 | wc
echo "Los estados de salida de cada comando son: ${PIPESTATUS[*]}"

### Comprobaciones en Bash

Antes de presentar las estructuras de control de flujo, es necesario saber como hacer comprobaciones en `Bash`.

Básicamente existen tres formas de hacer pruebas.

#### Comandos `test`,  `[` y `[[`

Los comandos `test` y `[` son sinónimos, con la excepción de que el comando `[` requiere un parámetro adicional (es obligatorio colocar «`]`» como último argumento) que no está presente en `test`. Todo lo demás es idéntico.

In [ ]:
type test; type [

Ambos comandos están definidos en las normas __POSIX__.

- `[[` es una palabra reservada. No forma parte del estándar  __POSIX__.
- Es más potente, amigable y rápido que `test` o que `[`.
- Es original de `ksh`, pero actualmente está soportado en `bash`.

[Aquí](https://mywiki.wooledge.org/BashFAQ/031) hay más información.

In [ ]:
type [[

Los tres funcionan de la misma manera. Realizan alguna comprobación y finalizan sin devolver nada por la salida estándar. Es en el estado de salida, donde se puede saber el resultado de la prueba. Si `$?` es `0` o `1`, la comprobación fue __verdadera__ o __falsa__ respectivamente.

__NOTA IMPORTANTE__

En todos los casos se tienen que respetar los espacios, entre los corchetes y su contenido, ya que que son comandos y parámetros. Así que se tienen que tratar como tal, ya que el espacio es el separador.

Los comandos `true` y `false`, siempre devuelven el estad de salida `0` y `1` respectivamente.

In [ ]:
type true; type false

__Ejemplo__

In [ ]:
#Comparando cadenas (orden lexicográfico).
#Se tiene que escapar «>», de lo contrario estaríamos redireccionando STDOUT.
test "Hola" \> "Holb"
echo $?

In [ ]:
test "Hola" > "Holb" #Crea un archivo vacío llamado Holb en el directorio actual.

In [ ]:
ls; rm Holb #Mostramos y seguidamente lo borramos.

In [ ]:
[ "Hola" \> "Holb" ]
echo $?

In [ ]:
[[ "Hola" > "Holb" ]] #Como [[ no es un comando, no es necesario escapar «>».
echo $?

In [ ]:
#Se tiene que escapar «<», de lo contrario estaríamos redireccionado STDIN.
test "Hola" \< "Holb"
echo $?

In [ ]:
test "Hola" < "Holb" #Redirecciona la entrada al archivo Holb, que ya borre.
#Si no lo hubiera borrado, el test se hubiera ejecutado correctamente, ya que
#al estar vacío el archivo, sólo se ejecuta test "Hola".
#El test hubiera devuelto 0, que es el valor esperado de la comparación correcta.
echo $?

In [ ]:
#Volvamos a crear el archivo para comprobarlo:
echo > Holb
test "Hola" < "Holb"
echo $?
rm Holb

In [ ]:
#Lo anterior equivale a:
test "Hola"
echo $?

In [ ]:
[ "Hola" \< "Holb" ] 
echo $?

In [ ]:
[[ "Hola" < "Holb" ]]
echo $?

In [ ]:
true #Siempre verdadero.
echo $?
false #Siempre falso.
echo $?

#### Cadenas

`test` o `[` | `[[` | Comprobación
:-- | :-- | :--
`\>` | `>` | Mayor que... (orden lexicográfico).
`\<` | `<` | Menor que... (orden lexicográfico).
`=` o `==` | `=` o `==` | Igual a...
`!=` | `!=` | Distinto de...
`-z` | `-z` | La cadena está vacía.
`-n` | `-n` | La cadena no está vacía.
No se puede | `=~` | La cadena a la izquierda de `=~`, cumple con alguna expresión regular a la derecha de `=~`.

Ejemplo con `[`:

In [ ]:
var1=aa; var2=" "; var3=aa; var4=""

echo 'var1=aa; var2=11; var3=aa; var4=""'
echo "-------------"

[ "$var1" = "$var2" ]
echo '[ "$var1" = "$var2" ]: ' $?
echo "-------------"
[[ $var1 == $var2 ]]
echo '[ "$var1" == "$var2" ]: '$?
echo "-------------"
[ $var1 == $var3 ]
echo '[ "$var1" == "$var3" ]: '$?
echo "-------------"
[[ "$var1" != "$var2" ]]
echo '[ "$var1" != "$var2" ]: '$?
echo "-------------"
[ "$var1" != "$var3" ]
echo '[ "$var1" != "$var3" ]: '$?
echo "-------------"
[ -z "$var2" ] #Espacio no es vacío.
echo '[ -z "$var2" ]: '$?
echo "-------------"
[ -z "$var4" ]
echo '[ -z "$var4" ]: '$?
echo "-------------"
[ -z "$var5" ] #var5, no fue definida. Esto equivale a estar vacía.
echo '[ -z "$var5" ]: '$?
echo "-------------"
[ -n "$var2" ]
echo '[ -n "$var2" ]: '$?
echo "-------------"
[ -n "$var4" ]
echo '[ -n "$var4" ]: '$?
echo "-------------"
[[ -n "$var5" ]]
echo '[ -n "$var5" ]: '$?

Ejemplo con `[[`:

In [ ]:
var1=aa; var2=" "; var3=aa; var4=""

echo 'var1=aa; var2=11; var3=aa; var4=""'
echo "-------------"

[[ $var1 = $var2 ]]
echo '[[ $var1 = $var2 ]]: ' $?
echo "-------------"
[[ $var1 == $var2 ]]
echo '[[ $var1 == $var2 ]]: '$?
echo "-------------"
[[ $var1 == $var3 ]]
echo '[[ $var1 == $var3 ]]: '$?
echo "-------------"
[[ $var1 != $var2 ]]
echo '[[ $var1 != $var2 ]]: '$?
echo "-------------"
[[ $var1 != $var3 ]]
echo '[[ $var1 != $var3 ]]: '$?
echo "-------------"
[[ -z $var2 ]] #Espacio no es vacío.
echo '[[ -z $var2 ]]: '$?
echo "-------------"
[[ -z $var4 ]]
echo '[[ -z $var4 ]]: '$?
echo "-------------"
[[ -z $var5 ]] #var5, no fue definida. Esto equivale a estar vacía.
echo '[[ -z $var5 ]]: '$?
echo "-------------"
[[ -n $var2 ]]
echo '[[ -n $var2 ]]: '$?
echo "-------------"
[[ -n $var4 ]]
echo '[[ -n $var4 ]]: '$?
echo "-------------"
[[ -n $var5 ]]
echo '[[ -n $var5 ]]: '$?

In [ ]:
var=abc
echo 'var=abc'
echo "-------------"
[[ $var =~ a ]]
echo '[[ $var =~ a ]]: '$?
echo "-------------"
[[ $var =~ b ]]
echo '[[ $var =~ b ]]: '$?
echo "-------------"
[[ $var =~ c ]]
echo '[[ $var =~ c ]]: '$?
echo "-------------"
[[ $var =~ d ]]
echo '[[ $var =~ d ]]: '$?
echo "-------------"
[[ $var =~ ^a ]]
echo '[[ $var =~ ^a ]]: '$?
echo "-------------"
[[ $var =~ ^b ]]
echo '[[ $var =~ ^b ]]: '$?
echo "-------------"
[[ $var =~ ^c ]]
echo '[[ $var =~ ^c ]]: '$?
echo "-------------"
[[ $var =~ a$ ]]
echo '[[ $var =~ a$ ]]: '$?
echo "-------------"
[[ $var =~ b$ ]]
echo '[[ $var =~ b$ ]]: '$?
echo "-------------"
[[ $var =~ c$ ]]
echo '[[ $var =~ c$ ]]: '$?
echo "-------------"
[[ $var =~ ab ]]
echo '[[ $var =~ ab ]]: '$?
echo "-------------"
[[ $var =~ ac ]]
echo '[[ $var =~ ac ]]: '$?
echo "-------------"
[[ $var =~ a.c ]]
echo '[[ $var =~ a.c ]]: '$?

In [ ]:
#Esto lo usamos para saber si una ruta estaba en la variable de entorno PATH.
[[ $PATH =~ /bin ]]
echo $?

Para saber más de expresiones regulares, pueden consultar [este tutorial](https://blog.desdelinux.net/con-el-terminal-uso-de-expresiones-regulares/#Entrando_en_materia) o [este otro](https://www.adictosaltrabajo.com/2015/01/29/regexsam/).

#### Enteros

`test` o `[` | `[[` | Comprobación
:-- | :-- | :--
`-lt` | `-lt` | Menor que... (orden en los enteros).
`-le` | `-le` | Menor o igual a... (orden lexicográfico).
`-eq` | `-eq` | Igual a...
`-ge` | `-ge` | Mayor o igual a...
`-gt` | `-gt` | Mayor que...
`-ne` | `-ne` | Desigual a...

In [ ]:
var1=1; var2=2; var3=3
echo 'var1=1; var2=2; var3=3'
echo "-------------"
[ $var1 -lt $var2 ]
echo '[ $var1 -lt $var2 ]: '$?
echo "-------------"
[ $var2 -lt $var1 ]
echo '[ $var2 -lt $var1 ]: '$?
echo "-------------"
[ $var2 -lt $var2 ]
echo '[ $var2 -lt $var2 ]: '$?
echo "-------------"
[ $var1 -gt $var2 ]
echo '[ $var1 -gt $var2 ]: '$?
echo "-------------"
[ $var2 -gt $var1 ]
echo '[ $var2 -gt $var1 ]: '$?
echo "-------------"
[ $var2 -gt $var2 ]
echo '[ $var2 -gt $var2 ]: '$?
echo "-------------"
[ $var1 -le $var2 ]
echo '[ $var1 -le $var2 ]: '$?
echo "-------------"
[ $var2 -le $var1 ]
echo '[ $var2 -le $var1 ]: '$?
echo "-------------"
[ $var2 -le $var2 ]
echo '[ $var2 -le $var2 ]: '$?
echo "-------------"
[ $var1 -ge $var2 ]
echo '[ $var1 -ge $var2 ]: '$?
echo "-------------"
[ $var2 -ge $var1 ]
echo '[ $var2 -ge $var1 ]: '$?
echo "-------------"
[ $var2 -ge $var2 ]
echo '[ $var2 -ge $var2 ]: '$?

__NOTA IMPORTANTE__

No se deben usar los operadores destinados a trabajar con cadenas, para comprobaciones numéricas.

En algunos casos, los enteros se pueden tratar como cadenas:

In [ ]:
[ 14 \> 12 ]
echo '[ 14 \> 12 ]: '$?
[ 14 \< 12 ]
echo '[ 14 \< 12 ]: '$?
[ 14 = 12 ]
echo '[ 14 = 12 ]: '$?
[ 12 = 12 ]
echo '[ 14 = 12 ]: '$?
[ 14 != 12 ]
echo '[ 14 != 12 ]: '$?

In [ ]:
[[ 14 > 12 ]]
echo '[[ 14 > 12 ]]: '$?
[[ 14 < 12 ]]
echo '[[ 14 < 12 ]]: '$?
[[ 14 = 12 ]]
echo '[[ 14 = 12 ]]: '$?
[[ 12 = 12 ]]
echo '[[ 12 = 12 ]]: '$?
[[ 14 != 12 ]]
echo '[[ 14 != 12 ]]: '$?

En otros no:

In [ ]:
[ 12 = 012 ]
echo '[ 11 = 012 ]: '$? #Las cadenas no son iguales.
[[ 12 = 012 ]]
echo '[[ 12 = 012 ]]: '$?
[ 12 -eq 012 ]
echo '[ 11 = 012 ]: '$? #No distingue entre enteros octales y enteros decimales.
[[ 12 -eq 012 ]]
echo '[[ 12 = 012 ]]: '$? #Un entero octal es distinto de un entero decimal.
echo '[[ 012 = 0012 ]]: '$? #Estos dos octales son iguales, pero como cadenas no.

Supongamos que quiero estar seguro de que el conjunto de permisos de mi directorio personal es el correcto (0775).

In [ ]:
#No funciona.
[ $(stat -c "%a" ~) = 0755 ]
echo $?
[[ $(stat -c "%a" ~) = 0755 ]]
echo $?

In [ ]:
[ $(stat -c "%a" ~) -eq 0755 ] #Funciona, pero no debería.
echo $?
[[ $(stat -c "%a" ~) -eq 0755 ]] #No funciona, porque stat, me lo da en enteros.
echo $?

Pero con números negativos no funciona el orden lexicográfico.

In [ ]:
[ -4 -lt -3 ]
echo $?
[ -4 \< -3 ]
echo $?
[[ -4 < -3 ]]
echo $?

#### Archivos

`test`, `[` y `[[` | Comprobación
:-- | :--
`-e`| Existe el archivo (no importa el tipo).
`-d`| Existe y es un directorio.
`-f`| Existe y es un archivo regular.
`-h` o `-L` | Existe y es un enlace simbólico.
`-ef` | El manual dice que es para comprobar si comparten el mismo inodo. Lo cierto es que da 0, si están enlazados (ya sea un enlace duro o simbólico).
`-s` | El archivo existe y no está vacío.

Hay muchas más comprobaciones, relacionadas con antigüedad, permisos, etc. Para más información consultar el manual.

In [ ]:
x-terminal-emulator --noclose -e 'man test' #Para ver el manual.

Creamos algunos archivos en `/tmp`.

In [ ]:
mkdir -p /tmp/dir1/dir2
echo "Hola" > /tmp/dir1/arch1
touch /tmp/dir1/arch2
ln -s /tmp/dir1/arch1 /tmp/dir1/arch1-sl
ln /tmp/dir1/arch1 /tmp/dir1/arch1-hl
tree --inodes -h /tmp/dir1

In [ ]:
#Esto funciona tanto con [ como con [[.
#Probaré sólo con [.

[ -e /tmp/dir1/arch1 ]
echo '[ -e /tmp/dir1/arch1 ]: '$?
[ -f /tmp/dir1/arch1 ] #Es un archivo regular.
echo '[ -f /tmp/dir1/arch1 ]: '$?
[ -d /tmp/dir1/arch1 ]
echo '[ -d /tmp/dir1/arch1 ]: '$?
[ -L /tmp/dir1/arch1 ]
echo '[ -L /tmp/dir1/arch1 ]: '$?
[ -h /tmp/dir1/arch1 ]
echo '[ -h /tmp/dir1/arch1 ]: '$?
[ -s /tmp/dir1/arch1 ] #No está vacío.
echo '[ -s /tmp/dir1/arch1 ]: '$?

In [ ]:
[ -s /tmp/dir1/arch2 ] #No tiene contenido.
echo '[ -s /tmp/dir1/arch2 ]: '$?

In [ ]:
[ -e /tmp/dir1 ]
echo '[ -e /tmp/dir1 ]: '$?
[ -f /tmp/dir1 ]
echo '[ -f /tmp/dir1]: '$?
[ -d /tmp/dir1 ] #Es un directorio.
echo '[ -d /tmp/dir11 ]: '$?
[ -L /tmp/dir1 ]
echo '[ -L /tmp/dir1/arch1 ]: '$?
[ -h /tmp/dir1 ]
echo '[ -h /tmp/dir1 ]: '$?
[ -s /tmp/dir1/arch1 ] #No está vacío.
echo '[ -s /tmp/dir1 ]: '$?

In [ ]:
#Los directorios nunca están vacíos, ya que siempre contienen a «.» y a «..».
[ -s /tmp/dir1/arch2 ] 
echo '[ -s /tmp/dir1/dir2 ]: '$?

In [ ]:
[ -e /tmp/dir1/arch1-sl ]
echo '[ -e /tmp/dir1/arch1-sl ]: '$?
[ -f /tmp/dir1/arch1-sl ] #Es un archivo regular.
echo '[ -f /tmp/dir1/arch1-sl ]: '$?
[ -d /tmp/dir1/arch1-sl ]
echo '[ -d /tmp/dir1/arch1-sl ]: '$?
[ -L /tmp/dir1/arch1-sl ] #Es un enlace simbólico.
echo '[ -L /tmp/dir1/arch1-sl ]: '$?
[ -h /tmp/dir1/arch1-sl ] #Es un enlace simbólico.
echo '[ -h /tmp/dir1/arch1-sl ]: '$?
[ -s /tmp/dir1/arch1-sl ] #No están vacíos, ya que contienen la referencia al original.
echo '[ -s /tmp/dir1/arch1-sl ]: '$?

In [ ]:
[ -e /tmp/dir1/arch3 ] #No creamos ningún arch3.
echo '[ -e /tmp/dir1/arch3 ]: '$?

In [ ]:
#Están enlazados.
[ /tmp/dir1/arch1 -ef /tmp/dir1/arch1-sl ] #No son enlaces duros.
echo '[ /tmp/dir1/arch1 -ef /tmp/dir1/arch1-sl ]: '$?
[ /tmp/dir1/arch1 -ef /tmp/dir1/arch1-hl ] #No son enlaces duros.
echo '[ /tmp/dir1/arch1 -ef /tmp/dir1/aarch1-hl ]: '$?
[[ /tmp/dir1/arch1 -ef /tmp/dir1/arch1-sl ]]
echo '[[ /tmp/dir1/arch1 -ef /tmp/dir1/arch1-sl ]]: '$?
[[ /tmp/dir1/arch1 -ef /tmp/dir1/arch1-hl ]]
echo '[[ /tmp/dir1/arch1 -ef /tmp/dir1/aarch1-hl ]]: '$?

#### Concatenando pruebas.

Para `test` existen los operadores `-a` (__and__) y `-o` (__or__) que ya casi están en desuso. Además se pueden agrupar expresiones con `\(` y `\)`.

In [ ]:
#1 < 2 o 2 < 1.
[ \( 1 -lt 2 \) -o \( 2 -lt 1 \) ]
echo $?

In [ ]:
#1 < 2 y 2 < 1.
[ \( 1 -lt 2 \) -a \( 2 -lt 1 \) ]
echo $?

Esta forma ya casi no se usa. En su lugar se suele hacer lo mismo de esta manera:

In [ ]:
#1 < 2 o 2 < 1.
[ 1 -lt 2 ] || [ 2 -lt 1 ]
echo $?

#1 < 2 y 2 < 1.
[ 1 -lt 2 ] && [ 2 -lt 1 ]
echo $?

También es posible agruparlos:

In [ ]:
#(1 < 2 y 2 < 1) o 1 = 1.
{ [ 1 -lt 2 ] && [ 2 -lt 1 ]; } || [ 1 -eq 1 ] #Es necesario el «;» antes del cierre «}».
echo $?

In [ ]:
#1 < 2 y (2 < 1 o 1 = -1).
[ 1 -lt 2 ] && { [ 2 -lt 1 ] || [ 1 -eq -1 ]; }
echo $?

Con `[[`, todo es más cómodo y natural.

In [ ]:
#1 < 2 o 2 < 1.
[[ 1 -lt 2 || 2 -lt 1 ]]
echo $?

#1 < 2 y 2 < 1.
[[ 1 -lt 2 &&  2 -lt 1 ]]
echo $?

#(1 < 2 y 2 < 1) o 1 = 1.
[[ ( 1 -lt 2 && 2 -lt 1 ) || 1 -eq 1 ]]
echo $?

#1 < 2 y (2 < 1 o 1 = -1).
[[ 1 -lt 2 && ( 2 -lt 1 || 1 -eq -1 ) ]]
echo $?

De ese modo se pueden combinar todo tipo de comprobaciones.

Para finalizar, con `!` se puede negar el test:

In [ ]:
[ 1 -lt 2 ]
echo $?
echo "-----------------"
test ! 1 -lt 2
echo $?
echo "-----------------"
[ ! 1 -lt 2 ]
echo $?
echo "-----------------"
[[ ! 1 -lt 2 ]]
echo $?

### Estructuras de control de flujo.

#### `if`

### Estructuras de control de flujo.

La estructura `if` funciona de manera similar a que en el resto de lenguajes.

```bash
if condición-1; then
    #Contenido si se cumple la con condición-1.
elif condición-2; then
    #Contenido si se cumple la con condición-2.
    .
    .
    .
elif condición-n; then
    #Contenido si se cumple la con condición-n.  
else
    #Contenido si no se cumple ninguna de las anteriores.
fi
```
`then` se puede colocar en la siguiente linea:

```bash
if condición-1
then
    #Contenido si se cumple la con condición-1.
elif condición-2
then
    #Contenido si se cumple la con condición-2.
    .
    .
    .
elif condición-n
then
    #Contenido si se cumple la con condición-n.  
else
    #Contenido si no se cumple ninguna de las anteriores.
fi
```
Los bloques `elif` y `else` son opcionales y se pueden usar tantos `elif` como sea necesario.

Ejemplo:

In [ ]:
#Se usa el estado de salida de la condición, para determinar el flujo.
if true; then
    echo hola
fi

if false; then
    echo nada de hola
fi

In [ ]:
#En casos sencillos, se puede usar la evaluación condicional.
#Es totalmente equivalente.
true && echo hola
false && echo nada de hola

In [ ]:
var="Saludo"

if [[ $var = "Saludo" ]] ; then
    echo hola
else
    echo nada de hola
fi

In [ ]:
var="No saludo aludo"

if [[ $var = "Saludo" ]] ; then
    echo hola
else
    echo nada de hola
fi

In [ ]:
#De igual manera:
var="Saludo"
[[ $var = "Saludo" ]] && echo hola || echo nada de hola

In [ ]:
var="No saludo aludo"
[[ $var = "Saludo" ]] && echo hola || echo nada de hola

In [ ]:
var="1"

if [[ $var = "1" ]] ; then
    echo "es 1"
elif [[ $var = "2" ]] ; then
    echo "es 2"
else
    echo "ni 1, ni 2"
fi

echo "-----------------"

var="2"

if [[ $var = "1" ]] ; then
    echo "es 1"
elif [[ $var = "2" ]] ; then
    echo "es 2"
else
    echo "ni 1, ni 2"
fi

echo "-----------------"

var="33"

if [[ $var = "1" ]] ; then
    echo "es 1"
elif [[ $var = "2" ]] ; then
    echo "es 2"
else
    echo "ni 1, ni 2"
fi

In [ ]:
#De igual forma:
var="1"
[[ $var = "1" ]] && echo "es 1" || { [[ $var = "2" ]] && echo "es 3" ]; } || echo "ni 1, ni 2"
echo "-----------------"
var="2"
[[ $var = "1" ]] && echo "es 1" || { [[ $var = "2" ]] && echo "es 3" ]; } || echo "ni 1, ni 2"
echo "-----------------"
var="33"
[[ $var = "1" ]] && echo "es 1" || { [[ $var = "2" ]] && echo "es 3" ]; } || echo "ni 1, ni 2"

In [ ]:
if mkdir /root/dir1 2>/dev/null; then
    echo "Sé se puede escribir en /root"
else
    echo "No se puede escribir en /root"
fi

In [ ]:
if mkdir /tmp/dir1/dir3 2>/dev/null; then
    echo "Sé se puede escribir en /tmp/dir1/dir3"
else
    echo "No se puede escribir en /tmp/dir1/dir3"
fi

In [ ]:
n_scp="scp1"

scp -i $n_scp << "fin"

if [[ $(id -u) != 0 ]]; then
   echo "El archivo necesita permiso de root."
   exit
fi

echo "No se muestra"

fin

exc $n_scp

In [ ]:
exc2(){
    x-terminal-emulator --noclose -e "sudo ./$1.sh"
}

n_scp="scp1"

scp $n_scp << "fin"

if [[ $(id -u) != 0 ]]; then
   echo "El archivo necesita permiso de root."
   exit
fi

echo "Se muestra sólo si es root."

fin

exc2 $n_scp

Este fue el código que se usó para modificar la variable __PATH__:

In [ ]:
#Modificado manualmente por el administrador
#Añade la ruta hacia julia.
if [[ ! $PATH =~ "/opt/julia/julia-1.5.2/bin" ]]; then
    PATH+=":/opt/julia/julia-1.5.2/bin"
fi

#### `case`

La estructura general de case es la siguiente.

```bash
case $var in
    varor1)
    #caso 1.
    ;;
    varor2 | valor 3)
    #caso 2.
    ;;
    «patron»)
    #caso 3.
    ;;
    varor4)
    #caso 4. Este caso no es excluyente, debido al «&».
    ;;&
    *)
    #En cualquier otro caso.
    ;; 
esac
```
Al llegar a un `;;`, se sale de la estructura, a menos que esté acompañado de un `&`. En ese caso continúa al siguiente bloque.

Supongamos que queremos comprobar la variable `numero`:

```bash
case $numero in
    1)
    echo "es 1"
    ;;
    2 | 3)
    echo "es 2 o 3"
    ;;
    1*) #Esto es una regularidad.
    echo "comienza con 1"
    ;;& #Esto provoca que siga hacia abajo.
    *5) #Esto es una regularidad.
     echo "termina con 5"
    ;;& #Esto provoca que siga hacia abajo.
    *)
    echo "Ni es 1, ni es 2, ni es 3."
    ;; 
esac
```
La almacenaré en una función, para evaluarla varias veces.

In [ ]:
mi_case(){
numero=$1 #Esto no es necesario, su puede poner directamente $1 en el case.
    echo "El número $numero:"
    case $numero in
        1)
        echo "es 1"
        ;;
        2 | 3)
        echo "es 2 o 3"
        ;;
        1*) #Esto es una regularidad.
        echo "comienza con 1"
        ;;& #Esto provoca que siga hacia abajo.
        *5) #Esto es una regularidad.
        echo "termina con 5"
        ;;& #Esto provoca que siga hacia abajo.
        *)
        echo "Ni es 1, ni es 2, ni es 3."
        ;; 
    esac
}

mi_case 1
echo "----------"
mi_case 2
echo "----------"
mi_case 3
echo "----------"
mi_case 4
echo "----------"
mi_case 10
echo "----------"
mi_case 15

In [ ]:
fun(){
    local info=false
    local ayuda=false
    local error=false
    
    case $1 in
        -i | --info)
        info=true
        shift #Se elimina el parámetro de tipo opción.
        ;;
        -h | --help)
        ayuda=true
        shift
        ;;
        -*)
        error=true
        ;;
        *)
        ;; 
    esac
   
    if $error; then
        echo "Opción no válida" >&2
        return 1
    fi
    
    if $ayuda; then
       echo "La ayuda es:"
   fi
   
    if $info; then
        echo "La info:"
    fi
    
    if [[ -z $1 ]]; then
        echo "No hay parámetros."
    else
        echo -e "Los parámetros son:\n$@"
    fi
}

fun -i
echo "----------"
fun --info
echo "----------"
fun -h
echo "----------"
fun --help
echo "----------"
fun --hep
echo "----------"
fun -i Hola nada

#### `for`

```bash
for var in «elemento iterable»; do
    #Por cada elemento del «elemento iterable»
    #var=«elemento»
done
```

In [ ]:
for var in uno dos tres; do
    echo $var
done

In [ ]:
#Es equivalente.
for var in "uno" "dos" "tres"; do
    echo $var
done

In [ ]:
for var in "uno dos tres"; do
    echo $var #Es un solo elemento.
done

In [ ]:
for var in {1..6}; do #Este es la otra forma de crear rangos.
    echo $var
done

In [ ]:
for ((var=1; var<=6; var++)); do #Esto se parece a «C».
    echo $var
done

In [ ]:
for var in $(seq 10 -2 1); do
    echo $var
done

In [ ]:
#Se puede usar cualquier comando.
for var in $(ls /tmp/dir1); do
    echo $var
done

In [ ]:
for var in ${arreglo[*]}; do
    echo $var
done

In [ ]:
for var in ${arreglo[@]}; do
    echo $var
done

In [ ]:
#Ahora se nota la diferencia entre «*» y «@»
for var in "${arreglo[*]}"; do
    echo $var
done
echo "-------------"
for var in "${arreglo[@]}"; do
    echo $var
done

Puedo salir de un bucle con `break`:

In [ ]:
break --help

Saltarme pasos con `continue`:

In [ ]:
continue --help

In [ ]:
for i in 1 2 3 4 5; do
    echo -e "\n"
    for j in 1 2 3 4 5; do
        echo -n " ($i, $j)"
    done
done

In [ ]:
for i in 1 2 3 4 5; do
    echo -e "\n"
    for j in 1 2 3 4 5; do
        if [[ $j = $i ]]; then
            break
        fi
        echo -n " ($i, $j)"
    done
done

In [ ]:
for i in 1 2 3 4 5; do
    echo -e "\n"
    for j in 1 2 3 4 5; do
        if [[ $j = $i ]]; then
            break 2
        fi
        echo -n " ($i, $j)"
    done
done

In [ ]:
for i in 1 2 3 4 5; do
    echo -e "\n"
    for j in 1 2 3 4 5; do
        if [[ $j = 3 ]]; then
            break 2
        fi
        echo -n " ($i, $j)"
    done
done

In [ ]:
for i in 1 2 3 4 5; do
    echo -e "\n"
    for j in 1 2 3 4 5; do
        if [[ $j = $i ]]; then
            continue
        fi
        echo -n " ($i, $j)"
    done
done

In [ ]:
for i in 1 2 3 4 5; do
    echo -e "\n"
    for j in 1 2 3 4 5; do
        if [[ $j = $i ]]; then
            continue 2 #En este caso coincide con break.
        fi
        echo -n " ($i, $j)"
    done
done

In [ ]:
for i in 1 2 3 4 5; do
    echo -e "\n"
    for j in 1 2 3 4 5; do
        if [[ $j = 3 ]]; then
            continue 2 #En este caso coincide con break.
        fi
        echo -n " ($i, $j)"
    done
done

echo -e "\n----------"

for i in 1 2 3 4 5; do
    echo -e "\n"
    for j in 1 2 3 4 5; do
        if [[ $j = 3 ]]; then
            break
        fi
        echo -n " ($i, $j)"
    done
done

In [ ]:
for i in 1 2 3 4 5; do
    echo -e "\n"
    for j in 1 2 3 4 5; do
        if [[ $j = $i ]]; then
            continue 2 #Ya no coinciden.
        fi
        echo -n " ($i, $j)"
    done
    echo -e "\nfin de la fila $i"
done

echo -e "\n----------"

for i in 1 2 3 4 5; do
    echo -e "\n"
    for j in 1 2 3 4 5; do
        if [[ $j = $i ]]; then
            break
        fi
        echo -n " ($i, $j)"
    done
    echo -e "\nfin de la fila $i"
done

#### `while`

```bash
while «condición»; do
    #Repite mientras se cumpla la condición.
done
```

In [ ]:
declare -i entero=0

while [[ entero -lt 10 ]]; do
    echo $entero
    entero+=1
done

In [ ]:
while ls -R /tmp; do #Se detiene cuando da error (Permiso denegado).
    echo $entero
    entero+=1
done

In [ ]:
n_scp="scp4"

scp $n_scp << "fin"

declare -i entero=0

while true; do
    echo $entero
    entero+=1
    sleep 1 #Se detiene por 1 segundo. 
done

fin

exc $n_scp

In [ ]:
n_scp="scp5"

scp $n_scp << "fin"

declare -i entero=0

while true; do
    echo $entero
    entero+=1
    sleep 1 #Se detiene por 1 segundo.
    
    if [[ $entero -eq 5 ]]; then
        break
    fi
done

fin

exc $n_scp

In [ ]:
n_scp="scp5"

scp $n_scp << "fin"

declare -i entero=0

while true; do

    entero+=1
    if [[ $entero -eq 4 ]]; then
        continue
    fi
    
    echo $entero
    sleep 1 #Se detiene por 1 segundo.
    
    if [[ $entero -eq 8 ]]; then
        break
    fi
done

fin

exc $n_scp

In [ ]:
fun(){
    local info=false
    local ayuda=false
    local error=false
    
    while [[ -n $1 ]]; do #Recorro todas las opciones.
        case $1 in
            -i | --info)
            info=true
            shift #Se elimina el parámetro de tipo opción.
            ;;
            -h | --help)
            ayuda=true
            shift
            ;;
            -*)
            error=true
            break #Sale del ciclo.
            ;;
            *)
            break #Sale del ciclo.
            ;; 
        esac
    done
   
    if $error; then
        echo "Opción no válida" >&2
        return 1
    fi
    
    if $ayuda; then
       echo "La ayuda es:"
   fi
   
    if $info; then
        echo "El informe es:"
    fi
    
    if [[ -z $1 ]]; then
        echo "No hay parámetros."
    else
        echo -e "Los parámetros son:\n$@"
    fi
}

fun -i
echo "----------"
fun --info
echo "----------"
fun -h
echo "----------"
fun --help
echo "----------"
fun --hep
echo "----------"
fun -i Hola nada

In [ ]:
fun -i -h Hola nada
echo "----------"
fun Hola nada

#### `until`

Funciona igual que `while`, pero con la condición negada:

In [ ]:
declare -i entero=0

while [[ entero -lt 10 ]]; do
    echo $entero
    entero+=1
done

In [ ]:
entero=0
until [[ ! entero -lt 10 ]]; do
    echo $entero
    entero+=1
done

In [ ]:
entero=0

until [[ entero -ge 10 ]]; do
    echo $entero
    entero+=1
done

#### `select` estructura de control y de entrada de datos

Supongamos que deseemos crear un menú de opciones. Se debe escoger una opción del menú para continuar con la ejecución del script. De no ser correcta se vuelve a pedir que se elija una opción.

Podemos combinar todo lo anterior para resolver el problema. Una posible solución sería:

In [ ]:
n_scp="for_read_echo"

scp $n_scp << "fin"

lista="Opción1 Opción2 Opción3" #Lista de opciones:
indices=1 #Lista de índices.

#Se muestra la lista de opciones:
echo "De la siguiente lista:"
declare -i i=1 #Iterador para la lista de índices.
for var in $lista; do
    echo "$i) $var: " #Mostramos la opción.
    i+=1
    indices+=" $i" #Añadimos el siguiente índice
done

#Se almacena el valor y se verifica que corresponda a una de las opciones.
#En caso contrario repetimos:
while true; do
    read -p "Elija una opción: " opcion #Almacenamos la respuesta.
    
    if [[ $indices =~ $opcion ]]; then #Si el índice está en la lista de índices.
        echo "Elegiste la opción $opcion."
        break #En la mayoría de los casos, debemos salir al elegir una opción correcta.
    else
        echo "Opción no válida." #Informamos del error y repetimos.
    fi
done

fin

exc $n_scp

La solución anterior es efectiva y esta bien construida. No obstante, contamos con una estructura de control de flujo (mezclada con una estructura de entrada de datos) que nos podría simplificar el diseño de los menús.

```bash
lista="Opción1 Opción2 .... OpciónN"
select var in $lista; do

    #Acción a realizar al elegir una opción $var de la lista.
    
done
```

In [ ]:
#Este sería un menú bastante simple.
#El menú muestra poca información y no se detiene nunca.
n_scp="select1"

scp $n_scp << "fin"

lista="Opción1 Opción2 Opción3"

select i in $lista; do
    echo "Elegiste la opción $i."
done

fin

exc $n_scp

Para cambiar el prompt por defecto de `selecet`  «`#?`», podemos usar la variable `PS3`.

In [ ]:
set #Muestra todos las variables y características declaradas actualmente.

In [ ]:
#Por ejemplo:
set | grep n_scp
set | grep PATH

In [ ]:
set | grep PS #Muestra las variables que conforman los distintos prompt. 

In [ ]:
#Este menú es más informativo, pero sigue sin detenerse.
n_scp="select2"

scp $n_scp << "fin"

lista="Opción1 Opción2 Opción3"

PS3="Elija una opción: " #Se cambia «#?» por «Elija una opción: » 
echo "De la siguiente lista:" #Un comentario previo a la presentación de la lista:

select i in $lista; do
    echo "Elegiste la opción $i."
done

fin

exc $n_scp

In [ ]:
#Ahora sí, es totalmente funcional.
n_scp="select3"

scp $n_scp << "fin"

lista="Opción1 Opción2 Opción3"

PS3="Elija una opción: " #Se cambia «#?» por «Elija una opción: » 
echo "De la siguiente lista:" #Un comentario previo a la presentación de la lista:

select i in $lista; do
#Si se elige una opción no listada, i="".
    if [[ -z $i ]]; then  #Revisamos si «i» no está vacía.
        echo "Opción no válida." #Informamos del error y repetimos.
    else
        echo "Elegiste la opción $i."
        break #En la mayoría de los casos debemos salir de select, al elegir una opción.
    fi
done

fin

exc $n_scp

Crear un menú para elegir el paquete a instalar.

El primer paso es obtener la lista de candidatos.

Supongamos que queremos buscar el paquete para instalar `julia` en el directorio de __descargas__ y en directorio actual.

In [ ]:
#Lista de los posibles paquetes a instalar.
paquete="julia*.tar.gz" #Patrón para buscar julia.
#Establecemos los directorios de búsqueda.
dir_instaladores="$(xdg-user-dir DOWNLOAD)" #directorio Descargas y el directorio actual.
lista_paquetes=($(find $dir_instaladores . -maxdepth 1 -iname "$paquete"))

#Podemos recorrer el arreglo para ver si funcionó.
for opcion in ${lista_paquetes[*]}; do
    echo $opcion
done
#Tiene problemas con los espacios.

Para evitar que se use el espacio como delimitador, se puede modificar la variable de entorno `IFS`.

In [ ]:
#Lista de los posibles paquetes a instalar.
paquete="julia*.tar.gz" #Patrón para buscar julia.
#Establecemos los directorios de búsqueda.
#IFS=$' '
dir_instaladores="$(xdg-user-dir DOWNLOAD)" #directorio «Descargas».
IFS=$'\n' #Cambiamos el valor de «' '» a «'\n'»

#Buscamos en «Descargas» y en el directorio actual.
lista_paquetes=($(find $dir_instaladores . -maxdepth 1 -iname "$paquete"))

#Podemos recorrer la el arreglo para ver si funcionó.
for opcion in ${lista_paquetes[*]}; do
    echo $opcion
done
IFS=$' ' #Regresamos al valor original

In [ ]:
echo ${lista_paquetes[0]} #Elegimos el primero.

Ya podemos construir el menú:

In [ ]:
n_scp="nenu_julia"

scp $n_scp << "fin"

#Se crea el arreglo con la opciones.
paquete="julia*.tar.gz"
dir_instaladores="$(xdg-user-dir DOWNLOAD)" #directorio «Descargas».
IFS=$'\n'

#Buscamos en «Descargas» y en el directorio actual.
lista_paquetes=($(find $dir_instaladores . -maxdepth 1 -iname "$paquete"))

PS3="Elija una opción: " #Se cambia «#?» por «Elija una opción: » 
echo "De la siguiente lista:"
select i in ${lista_paquetes[*]}; do
    
#Si se elige una opción no listada, i="".
    if [[ -z $i ]]; then  #Revisamos si i no está vacía.
        echo "Opción no válida." #Informamos del error y repetimos.
    else
        echo -e "Se instlará el paquete:\n$i."
        break #En la mayoría de los casos debemos salir de select, al elegir una opción.
    fi
done
IFS=$' ' #Regresamos al valor original

fin

exc $n_scp

Si el script anterior se ejecuta como superusuario, puede tener el siguiente problema.

In [ ]:
#Devuelve el directorio de descargas de root.
n_scp="select3"

scp $n_scp << "fin"

sudo xdg-user-dir DOWNLOAD

fin

exc $n_scp

Dentro de un proceso lanzado por un script con `sudo`, podemos usar la variable `SUDO_USER` para saber cual usuario inició el proceso.

Con el comando `runuser`, se pueden lanzar procesos como si fuéramos otro usuario. Este comando no pide contraseña, ya que es obligatorio ser `root` para ejecutarlo.

```bash
echo $SUDO_USER #Usuario que lanzó el proceso como root.
whoami #Usuario actual «root».
#Devuelve el directorio «Descargas» del usuario que lanzó el proceso con «sudo».
runuser -s /bin/bash $SUDO_USER -c "xdg-user-dir DOWNLOAD"
```

### Diálogos interactivos desde la linea de comando.

Se pueden crear diálogos similares a una interfaz gráfica.

Los siguientes comandos se usan para este propósito:

- `whiptail`
  - Está instalado por defecto en Debian y derivadas.
  - Por esa razón, muchos paquetes lo usan en Ubuntu.
  - [Aquí](https://www.raulprietofernandez.net/blog/shells/como-crear-cuadros-de-dialogo-en-tus-scripts-de-gnu-linux) se puede ver un tutorial básico.
  - [Este](https://sio2sio2.github.io/doc-linux/03.scripts/06.misc/05.whiptail.html) Es uno más avanzado. Debe leerse después del anterior.
  - El diálogo `--infobox` no funciona con la terminal `xterm`. Yo personalmente la cambio a `linux` (`TERM=linux`). Esto lo expliqué el la primera plática.
  - El diálogo `--inputbox` necesita conmutar la salida estándar con la salida de error estándar (`3>&1 1>&2 2>&3`). De lo contrario no funciona.
- `Dialog`
  - Es más atractivo, pero no está instalado por defecto.
  - Lo desarrolla Red Hat.
  - [Esta](https://aplicacionesysistemas.com/dialog-crear-menus-tus-scripts/) es una introducción.

### Miniproyecto (Instalador para el taller)

- Presentación en directo.
- Se entrega en un directorio adjunto al notebook.